# TM10007 Assignment: Prediction of Tumor Grade in Brain Cancer

Group 4: Kiefer Comassi (4402359), Myrthe van Haaften (4547470), Frédérique Koopman (4470885), Stephanie Stoutjesdijk (4557808)


## Installing and importing functions and packages

In [21]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git
#!pip install missingpy

In [22]:
import numpy as np
import pandas as pd
from math import floor
import matplotlib.pyplot as plt

from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from missingpy import KNNImputer
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV, SelectKBest, f_classif, SelectFromModel
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

from brats.load_data import load_data

## Loading and splitting data 

In [23]:
# Load data
data = load_data()

# Splitting feature values and patient labels
FEATURES = data.drop(columns=['label'])
LABELS = data['label']

GBM = FEATURES.loc[LABELS=='GBM']
LGG = FEATURES.loc[LABELS=='LGG']


## Preprocessing before crossvalidation

Overview of the NaN's in the dataset

In [24]:
# Determining the number of NaN's
NO_NAN_ROW_TOTAL = FEATURES.isnull().sum(axis=1)             # Number of NaN's per patient for GBM and LGG patients
NO_NAN_COL_TOTAL = FEATURES.isnull().sum(axis=0)             # Number of NaN's per feature for GBM and LGG patients

GBM_NO_NAN_COL = GBM.isnull().sum(axis=0)                    # Number of NaN's per feature for GBM patients
LGG_NO_NAN_COL = LGG.isnull().sum(axis=0)                    # Number of NaN's per feature for LGG patients
OVERVIEW_NAN = { 'Total': NO_NAN_COL_TOTAL, 'GBM': GBM_NO_NAN_COL, 'LGG': LGG_NO_NAN_COL } 


Feature selection based on the number of NaN's. Threshold = the maximum number of NaN's in a column

In [25]:
# Define percentage of patients with no data for a certain feature, above which the feature is discarded
PERC_MISSING_GBM = 30
PERC_MISSING_LGG = 30

# Determining threshold for discarding feature based on above percentage
THRESHOLD_GBM = floor((PERC_MISSING_GBM/100) * len(GBM.index))
THRESHOLD_LGG = floor((PERC_MISSING_LGG/100) * len(LGG.index))

# Initialisation
FEATURES_REMOVED = []

# Append names of features that should be discarded to list

for feature in GBM_NO_NAN_COL[GBM_NO_NAN_COL > THRESHOLD_GBM].index[:]:
    FEATURES_REMOVED.append(feature)

for feature in LGG_NO_NAN_COL[LGG_NO_NAN_COL > THRESHOLD_LGG].index[:]:
    FEATURES_REMOVED.append(feature)

# Remove features from dataset
DATA_FEAT_SEL = FEATURES.drop(columns=[features for features in set(FEATURES_REMOVED)])

# The variables (series) below 'bins' the NaN's:
# - the index column is the amount of NaN's in the dataset 
# - the second column is the amount of features that have this amount of NaN's

#aantal_NAN_GBM = GBM_no_nan_col.value_counts()
#aantal_NAN_LGG = LGG_no_nan_col.value_counts()
#aantal_NAN_total = no_nan_col.value_counts()


Patient selection based on the number of NaN's. Threshold = the maximum number of NaN's in a row.

In [26]:
# Patient selection 

# Percentage/number of features that a patient is allowed to miss. When above this amount, this patient is removed from the trainingset, because it is missing too many features. 
PERC_MISSING_SAMPLE = 30

# Make the threshold
THRESHOLD_SAMPLE = floor((PERC_MISSING_SAMPLE/100) * len(DATA_FEAT_SEL.columns))

# Number of NaN's per patient AFTER removing some features
NO_NAN_ROW_TRAIN = DATA_FEAT_SEL.isnull().sum(axis=1)      

# Make an empty list, which will be filled with samples that are above the threshold
SAMPLES_REMOVED = [] 
LABELS_SEL = LABELS

# Looping over the trainingset to determine which patients are above the threshold, and remove them directly.
print('These samples are removed from dataset:')
for sample in NO_NAN_ROW_TRAIN[NO_NAN_ROW_TRAIN > THRESHOLD_SAMPLE].index[:]:
    if sample:
        print(sample)
        SAMPLES_REMOVED.append(sample)      # This should be removed in combination with the comment before
        DATA_FEAT_SEL = DATA_FEAT_SEL.drop(index=sample)
        LABELS_SEL = LABELS_SEL.drop(index=sample)

print(f'{len(DATA_FEAT_SEL)}/{len(FEATURES)} samples are left in dataset')

These samples are removed from dataset:
TCGA-HT-7686
166/167 samples are left in dataset


Evaluation of data distribution and outliers

In [27]:
# Evaluate data distribution: 
IMPUTER_GBM = KNNImputer(n_neighbors=5, weights="uniform")
IMPUTER_LGG = KNNImputer(n_neighbors=5, weights="uniform")        

X_GBM = DATA_FEAT_SEL[LABELS_SEL=='GBM']        
X_IMP_GBM = IMPUTER_GBM.fit_transform(X_GBM)

X_LGG = DATA_FEAT_SEL[LABELS_SEL=='LGG']        
X_IMP_LGG = IMPUTER_LGG.fit_transform(X_LGG)   

NO_NON_NORMAL_GBM = 0
FEAT_NON_NORM_GBM = list()
for index, feature in enumerate(X_IMP_GBM.T):
    t_stat, p_value = shapiro(feature)
    if p_value<0.05:
        NO_NON_NORMAL_GBM += 1 
        FEAT_NON_NORM_GBM.append(index)

NO_NON_NORMAL_LGG = 0
FEAT_NON_NORM_LGG = list()
for index, feature in enumerate(X_IMP_LGG.T):
    t_stat, p_value = shapiro(feature)
    if p_value<0.05:
        NO_NON_NORMAL_LGG += 1 
        FEAT_NON_NORM_LGG.append(index)



In [28]:

# Evaluate per GBM patient each feature
NO_OUTLIERS_GBM = 0

for patient in X_IMP_GBM:
    feat_out = 0
    for index, feature in enumerate(patient):
        q25, q75 = np.percentile(X_IMP_GBM[:,index], 25), np.percentile(X_IMP_GBM[:,index], 75)
        iqr = q75 - q25
        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off
        outlier = ((feature>upper) | (feature<lower))
        feat_out += outlier
    if feat_out > 70:
        NO_OUTLIERS_GBM += 1

# Evaluate per LGG patient each feature
NO_OUTLIERS_LGG = 0

for patient in X_IMP_LGG:
    feat_out = 0
    for index, feature in enumerate(patient):
        q25, q75 = np.percentile(X_IMP_LGG[:,index], 25), np.percentile(X_IMP_LGG[:,index], 75)
        iqr = q75 - q25
        cut_off = iqr * 1.5
        lower, upper = q25 - cut_off, q75 + cut_off
        outlier = ((feature>upper) | (feature<lower))
        feat_out += outlier
    if feat_out > 70:
        NO_OUTLIERS_LGG += 1

# Calculate mean and standard deviation per feature based on training data
#MEAN = DATA_TOTAL_IMPUTED.mean(axis=0)
#STD = DATA_TOTAL_IMPUTED.std(axis=0)

# Evaluate per patient each feature
#NO_OUTLIERS = 0
#for patient in DATA_TOTAL_IMPUTED:
#    feat_out = 0
#    for feature, mu, std in zip(patient, MEAN, STD):
#        outlier = ((feature>mu+3*std) | (feature<mu-3*std))
#        feat_out += outlier
#    if feat_out > 30:
#        no_outliers += 1

## Function definitions

In [29]:
# Imputation

def knn_impute_train_set(X_train, X_test):
    # Definition of imputers
    imputer = KNNImputer(n_neighbors=5, weights="uniform")
    
    # Impute train and test set 
    imputer.fit(X_train)
    X_train_imp = imputer.transform(X_train)
    X_test_imp = imputer.transform(X_test)
    return X_train_imp, X_test_imp

In [30]:
# Scaling

def scale_train_and_test_data(X_train, X_test):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scal = scaler.transform(X_train)
    X_test_scal = scaler.transform(X_test)
    return X_train_scal, X_test_scal

In [31]:
# Feature selection using univariate feature selection

def select_features_univariate(X_train, y_train, X_test):
    selection_method = SelectKBest(f_classif, k=20).fit(X_train, y_train)
    X_train_sel = selection_method.transform(X_train)
    X_test_sel = selection_method.transform(X_test)
    return X_train_sel, X_test_sel, selection_method.get_support(indices=True)


In [32]:
# Feature selection using L1 
def select_features_L1(X_train, y_train, X_test):
    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X_train, y_train)
    model = SelectFromModel(lsvc, prefit=True, threshold = 'median')
    X_train_sel = model.transform(X_train)
    X_test_sel = model.transform(X_test)
    return X_train_sel, X_test_sel, model.get_support(indices=True)

In [33]:
# Feature selection using recursive feature elimination

def select_features_rfecv(X_train, y_train):

    svc = svm.SVC(kernel="linear")
    optimal_number_features = list()

    # classifications
    rfecv = RFECV(
        estimator=svc, step=1, 
        cv=StratifiedKFold(4),
        scoring='roc_auc')
    rfecv.fit(X_train, y_train)                               

    optimal_number_features.append(rfecv.n_features_)
    print("Optimal number of features : %d" % rfecv.n_features_)

    # Plot number of features VS. cross-validation scores
    plt.figure()
    plt.xlabel("Number of features selected")
    plt.ylabel("Cross validation score (nb of correct classifications)")
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    plt.show()

    return rfecv.support_
    #print(rfecv.ranking_)
    #np.absolute(rfecv.estimator_.coef_)

In [34]:
def pca_analysis(X_train, X_test):
    pca = PCA(n_components=20)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

In [35]:
# Randomized Search for Random Forrest Classifier

def rf_randomized_search(X_train, y_train):
    '''Perform a Randomized Search on the training set 
    to find the optimal hyperparameters.
    Input: training data and labels
    Output: Random Forest Classifier with optimal hyperparameters'''

    parameters = {
        'n_estimators': [32, 64, 128, 150],
        'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
        'min_samples_split': [2, 4, 6],
        'max_depth': [1,6,15,28,32]
    }

    grid = RandomizedSearchCV(RandomForestClassifier(), parameters, refit=True, verbose=0, n_iter = 20)
    grid.fit(X_train, y_train)

    rf_classifier = grid.best_estimator_
    #print(rf_classifier)

    return rf_classifier


In [36]:
# Randomized Search for K-Nearest Neighbors Classifier

def knn_randomized_search(X_train, y_train):
    '''Perform a Randomized Search on the training set 
    to find the optimal hyperparameters.
    Input: training data and labels
    Output: K-Nearest Neighbors Classifier with optimal hyperparameters'''

    parameters = {  'n_neighbors': list(range(3,31)),
                    'weights': ["uniform", "distance"]
                    }

    grid = RandomizedSearchCV(KNeighborsClassifier(), parameters, refit=True, verbose=0, n_iter = 20)
    grid.fit(X_train, y_train)

    knn_classifier = grid.best_estimator_
    #print(knn_classifier)

    return knn_classifier


In [37]:
# Randomized Search for Support Vector Machine Classifier

def svm_randomized_search(X_train, y_train):
    '''Perform a Randomized Search on the training set 
    to find the optimal hyperparameters.
    Input: training data and labels
    Output: SVM Classifier with optimal hyperparameters'''

    parameters =  {'kernel': ['linear', 'rbf', 'poly'],
                    'C': [0.01, 0.1, 1, 10, 100], 
                    'gamma': [1, 0.1, 0.01, 0.001, 'auto', 'scale'],
                    'degree': [1, 2, 3, 4, 5],
                    'coef0': [0.01, 0.5, 1, 5, 10, 20]} 

    grid = RandomizedSearchCV(SVC(), parameters, refit=True, verbose=0, n_iter = 20)

    grid.fit(X_train, y_train)

    svm_classifier = grid.best_estimator_
    #print(svm_classifier)

    return svm_classifier


In [38]:
def get_accuracy_classifier(classifier, X_train, y_train, X_test, y_test):

    classifier.fit(X_train, y_train)
    prediction = classifier.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, prediction)
    return accuracy

## Crossvalidation 

In [39]:

skf = StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

ACCURACIES_OUTER = list()
FEATURES_SELECTED_TOTAL = list()
fold = 0
PERFORMANCE = pd.DataFrame(columns=['SVM', 'RF', 'KNN'])

for train_index, test_index in skf.split(DATA_FEAT_SEL, LABELS_SEL):
    fold += 1 
    print(f'Run {fold} of outer crossvalidation')
    X_train, X_test = np.array(DATA_FEAT_SEL)[train_index], np.array(DATA_FEAT_SEL)[test_index]
    y_train, y_test = np.array(LABELS_SEL)[train_index], np.array(LABELS_SEL)[test_index]

    gbm_train = X_train[y_train=='GBM']
    lgg_train = X_train[y_train=='LGG']

    # ---------------------------------- IMPUTATION -----------------------------------

    X_train_imp, X_test_imp = knn_impute_train_set(X_train, X_test)
 
    # ------------------------------------ SCALING ----------------------------------------
    X_train_scal, X_test_scal = scale_train_and_test_data(X_train_imp, X_test_imp)

    # ------------------------------------ FEATURE SELECTION -------------------------------
    #RFECV
    #selected_features = select_features_rfecv(X_train_scal, y_train)
    #X_train_sel = X_train_scal[:,selected_features]
    #X_test_sel = X_test_scal[:,selected_features]

    # Univariate and L1
    X_train_sel, X_test_sel, selected_indices = select_features_univariate(X_train_scal, y_train, X_test_scal)
    #X_train_sel, X_test_sel, selected_indices = select_features_L1(X_train_scal, y_train, X_test_scal)
    print(selected_indices)
    features_selected = list()
    for index in selected_indices:
        features_selected.append(DATA_FEAT_SEL.columns[index])
    FEATURES_SELECTED_TOTAL.append(features_selected)

    #X_train_sel, X_test_sel = pca_analysis(X_train_scal, X_test_scal)

    # -------------- INNER CROSSVALIDATION HYPERPARAMETERS---------------------------------
    svm_classifier = svm_randomized_search(X_train_sel, y_train)   

    rf_classifier = rf_randomized_search(X_train_sel, y_train)

    knn_classifier = knn_randomized_search(X_train_sel, y_train)

    clfs = [svm_classifier, rf_classifier, knn_classifier]
    clfs_names = ['SVM', 'RF', 'KNN']

    # ---------------- OUTER CROSSVALIDATION------------------------------------------------
    acc_dict = dict()
    accuracy_total = list()

    for clf, name in zip(clfs, clfs_names):
        acc_dict[name] = get_accuracy_classifier(clf, X_train_sel, y_train, X_test_sel, y_test)
    accuracy_total.append(acc_dict)
    PERFORMANCE = PERFORMANCE.append(acc_dict, ignore_index=True)
    print(acc_dict)
    ACCURACIES_OUTER.append(accuracy_total)


    



Run 1 of outer crossvalidation
[  0   1   7   9  10  12  13  16  17 131 161 164 197 200 290 384 387 437
 439 468]
{'SVM': 0.7647058823529411, 'RF': 0.8529411764705882, 'KNN': 0.7941176470588235}
Run 2 of outer crossvalidation
[  0   1   9  10  12  13  16  17 131 157 161 163 235 237 336 384 437 439
 468 473]
{'SVM': 0.8484848484848485, 'RF': 0.8787878787878788, 'KNN': 0.8787878787878788}
Run 3 of outer crossvalidation
[  0   1   8   9  10  12  13  16  17 131 161 164 170 235 237 252 339 384
 387 468]
{'SVM': 0.9090909090909091, 'RF': 0.9090909090909091, 'KNN': 1.0}
Run 4 of outer crossvalidation
[  0   1   8   9  10  12  13  16  17 116 131 157 161 164 237 384 387 388
 437 468]
{'SVM': 0.8787878787878788, 'RF': 0.8787878787878788, 'KNN': 0.8787878787878788}
Run 5 of outer crossvalidation
[  0   1   8   9  10  12  13  16  17 131 161 170 173 206 209 237 303 384
 387 468]
{'SVM': 0.9090909090909091, 'RF': 0.9090909090909091, 'KNN': 0.9090909090909091}


In [40]:
print(PERFORMANCE)
MEAN_ACC = PERFORMANCE.mean()
print(MEAN_ACC)

SVM        RF       KNN
0  0.764706  0.852941  0.794118
1  0.848485  0.878788  0.878788
2  0.909091  0.909091  1.000000
3  0.878788  0.878788  0.878788
4  0.909091  0.909091  0.909091
SVM    0.862032
RF     0.885740
KNN    0.892157
dtype: float64
